# Generate invalid_documents column for each query

In [15]:
import pandas as pd
import random

# Load the CSV file
file_path = 'emath_qns_eval_data.csv'
df = pd.read_csv(file_path)

# Ensure 'valid_documents' and 'invalid_documents' columns are strings for processing
df['valid_documents'] = df['valid_documents'].astype(str)
if 'invalid_documents' not in df.columns:
    df['invalid_documents'] = ''

# Extract all unique docIDs from the valid_documents column
all_doc_ids = set()
for docs in df['valid_documents']:
    doc_ids = [doc.strip() for doc in docs.split(';') if doc.strip()]
    all_doc_ids.update(doc_ids)

print(all_doc_ids)


{"'674dcd9fb00b977d048c9306'", "'674dce98b00b977d048c93dc'", "'674dcdccb00b977d048c932a'", "'674dcdb4b00b977d048c931a'", "'674dce23b00b977d048c93b5'", "'674dce19b00b977d048c93a1'", "'674dcd9fb00b977d048c9305'", "'674dcddfb00b977d048c934f'", "'674dcd9db00b977d048c9300'", "'674dcddfb00b977d048c9350'", "'674dcde3b00b977d048c9357'", "'674dcdb4b00b977d048c931b'", "'674dce0fb00b977d048c9395'", "'674dce8db00b977d048c93c7'", "'674dcdd3b00b977d048c9337'", "'674dcdccb00b977d048c932b'", "'674dcd9cb00b977d048c92ff'", "'674dcde4b00b977d048c9358'", "'674dcd9eb00b977d048c9304'", "'674dce93b00b977d048c93d2'", "'674dce1ab00b977d048c93a4'", "'674dce93b00b977d048c93d3'", "'674dcdd3b00b977d048c9338'", "'674dcde1b00b977d048c9352'", "'674dce8eb00b977d048c93ca'", "'674dce25b00b977d048c93b9'", "'674dcdb5b00b977d048c931d'", "'674dce10b00b977d048c9397'", "'674dce0cb00b977d048c938f'", "'674dce18b00b977d048c939f'", "'674dcddeb00b977d048c934d'", "'674dcde3b00b977d048c9356'"}


In [16]:
# Clean and populate the 'invalid_documents' column with random docIDs
for index, row in df.iterrows():
    # Strip whitespace and create a set of valid document IDs
    valid_docs = set(doc.strip() for doc in row['valid_documents'].split(';') if doc.strip())
    print("Valid docs: ", valid_docs)
    
    # Create a pool of invalid document IDs, excluding the valid ones
    invalid_docs_pool = list(all_doc_ids - valid_docs)  # Exclude valid documents from the pool
    print("Invalid docs pool: ", invalid_docs_pool)
    
    # Randomly select a number of invalid documents
    if invalid_docs_pool:  # Ensure there are invalid docs to choose from
        num_invalid_docs = random.randint(1, len(valid_docs)-1)  # Random number of invalid docs, up to the number of valid docs
        invalid_docs = random.sample(invalid_docs_pool, num_invalid_docs)
        df.at[index, 'invalid_documents'] = '; '.join(invalid_docs)
    else:
        # If no invalid docs are available, leave the column empty
        df.at[index, 'invalid_documents'] = ''


# Save the updated dataframe to a new CSV file
output_path = 'emath_qns_eval_data_updated.csv'
df.to_csv(output_path, index=False)

output_path

Valid docs:  {"'674dcdb4b00b977d048c931b'", "'674dce0fb00b977d048c9395'", "'674dce10b00b977d048c9397'", "'674dcde1b00b977d048c9352'", "'674dce93b00b977d048c93d2'", "'674dce93b00b977d048c93d3'"}
Invalid docs pool:  ["'674dcd9fb00b977d048c9306'", "'674dce98b00b977d048c93dc'", "'674dcdccb00b977d048c932a'", "'674dcdb4b00b977d048c931a'", "'674dce23b00b977d048c93b5'", "'674dce19b00b977d048c93a1'", "'674dcd9fb00b977d048c9305'", "'674dcddfb00b977d048c934f'", "'674dcd9db00b977d048c9300'", "'674dcddfb00b977d048c9350'", "'674dcde3b00b977d048c9357'", "'674dce8db00b977d048c93c7'", "'674dcdd3b00b977d048c9337'", "'674dcdccb00b977d048c932b'", "'674dcd9cb00b977d048c92ff'", "'674dcde4b00b977d048c9358'", "'674dcd9eb00b977d048c9304'", "'674dce1ab00b977d048c93a4'", "'674dcdd3b00b977d048c9338'", "'674dce8eb00b977d048c93ca'", "'674dce25b00b977d048c93b9'", "'674dcdb5b00b977d048c931d'", "'674dce0cb00b977d048c938f'", "'674dce18b00b977d048c939f'", "'674dcddeb00b977d048c934d'", "'674dcde3b00b977d048c9356'"]
Valid

'emath_qns_eval_data_updated.csv'

# Verify that docIDs in invalid_documents are not present in valid_documents

In [20]:
df = pd.read_csv('emath_qns_eval_data_updated.csv')

# Validate that "invalid_documents" do not contain any docIDs from "valid_documents"
validation_errors = []

for index, row in df.iterrows():
    valid_docs = set(doc.strip() for doc in row['valid_documents'].split(';') if doc.strip())
    invalid_docs = set(doc.strip() for doc in row['invalid_documents'].split(';') if doc.strip())
    
    # Check if there is any overlap between valid and invalid documents
    if valid_docs & invalid_docs:  # Intersection should be empty
        validation_errors.append((index, valid_docs & invalid_docs))  # Record row index and overlapping docIDs

# Display validation results
if validation_errors:
    print(f"Validation errors found in {len(validation_errors)} rows:")
    for error in validation_errors:
        print(f"Row {error[0]} has overlapping docIDs: {error[1]}")
else:
    print("Validation passed: No overlapping docIDs between valid_documents and invalid_documents.")

Validation passed: No overlapping docIDs between valid_documents and invalid_documents.


# Add queries with random questions not related to math questions

In [4]:
import pandas as pd
import random

# Reload the existing CSV file (since the environment was reset)
file_path = 'emath_qns_eval_data_updated.csv'
df = pd.read_csv(file_path)

# Generate new unrelated queries
new_queries = [
    "What is the periodic table and who invented it?",
    "Explain the water cycle with examples.",
    "What are the different states of matter?",
    "What is the difference between velocity and speed?",
    "How do cells divide during mitosis?",
    "What was the significance of the Industrial Revolution?",
    "Who were the main leaders during World War II?",
    "What is the Magna Carta and why is it important?",
    "Describe the fall of the Roman Empire.",
    "Who built the Great Wall of China?",
    "What are the themes of George Orwell's '1984'?",
    "Who wrote 'Pride and Prejudice,' and what is its plot?",
    "What is the symbolism in Robert Frost's poem 'The Road Not Taken'?",
    "Can you analyze the character of Macbeth in Shakespeare's play?",
    "What is the significance of the opening line in 'Moby-Dick'?",
    "What are the seven wonders of the ancient world?",
    "How does a bill become law in the United States?",
    "What is the United Nations and what are its functions?",
    "Explain the concept of democracy.",
    "What is the main purpose of the International Space Station?",
    "What is the difference between a simile and a metaphor?",
    "How do airplanes stay in the air?",
    "What are the main ingredients in baking a cake?",
    "What is the Fibonacci sequence used for in real life?",
    "What is the history of the internet?"
]


# Extract all unique docIDs from the valid_documents column in the loaded data
all_doc_ids = set()
for docs in df['valid_documents'].dropna():
    doc_ids = [doc.strip() for doc in str(docs).split(';') if doc.strip()]
    all_doc_ids.update(doc_ids)

# Create new rows for these unrelated queries
new_rows = []
for query in new_queries:
    # Sample invalid documents
    num_invalid_docs = random.randint(1, 5)  # Randomly choose the number of invalid documents
    invalid_docs = random.sample(all_doc_ids, num_invalid_docs)
    invalid_docs_field = '; '.join(invalid_docs)

    # Create a new row
    new_rows.append({
        "question_type": "Random",
        "question": query,
        "valid_documents": "",
        "invalid_documents": invalid_docs_field
    })

# Convert new rows into a DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Append new rows to the existing DataFrame
df_updated = pd.concat([df, new_rows_df], ignore_index=True)
display(df_updated)


/var/folders/__/sxrs_frj41bdj4fb9smfcr9m0000gn/T/ipykernel_81313/1483437277.py:49: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  invalid_docs = random.sample(all_doc_ids, num_invalid_docs)


,question_type,question,valid_documents,invalid_documents
0,Multiplication of 2 matrices,Can you provide practice problems on multiplyi...,'674dcdb4b00b977d048c931b'; '674dcde1b00b977d0...,'674dcde3b00b977d048c9356'; '674dcd9fb00b977d0...
1,Multiplication of 2 matrices,I need some exercises involving matrix multipl...,'674dcdb4b00b977d048c931b'; '674dcde1b00b977d0...,'674dcde4b00b977d048c9358'
2,Multiplication of 2 matrices,Where can I find practice questions on multipl...,'674dcdb4b00b977d048c931b'; '674dcde1b00b977d0...,'674dce98b00b977d048c93dc'; '674dcdb4b00b977d0...
3,Multiplication of 2 matrices,Do you have any sample questions on multiplyin...,'674dcdb4b00b977d048c931b'; '674dcde1b00b977d0...,'674dcddfb00b977d048c9350'
4,Multiplication of 2 matrices,Could you give me some problems to practice ma...,'674dcdb4b00b977d048c931b'; '674dcde1b00b977d0...,'674dcdb4b00b977d048c931a'; '674dcdb5b00b977d0...
5,Finding the mean median and mode,Can you provide practice problems on finding t...,'674dcd9cb00b977d048c92ff'; '674dcd9db00b977d0...,'674dcdb4b00b977d048c931b'; '674dce0fb00b977d0...
6,Finding the mean median and mode,"I need exercises involving calculating mean, m...",'674dcd9cb00b977d048c92ff'; '674dcd9db00b977d0...,'674dcdb5b00b977d048c931d'; '674dcde1b00b977d0...
7,Finding the mean median and mode,"Where can I find practice questions on mean, m...",'674dcd9cb00b977d048c92ff'; '674dcd9db00b977d0...,'674dce18b00b977d048c939f'; '674dce25b00b977d0...
8,Finding the mean median and mode,Do you have any sample questions on finding me...,'674dcd9cb00b977d048c92ff'; '674dcd9db00b977d0...,'674dcdb4b00b977d048c931a'; '674dcd9fb00b977d0...
9,Finding the mean median and mode,Could you give me some problems to practice ca...,'674dcd9cb00b977d048c92ff'; '674dcd9db00b977d0...,'674dce23b00b977d048c93b5'; '674dce18b00b977d0...


In [5]:
# Save the updated DataFrame to a new CSV file
output_path_updated = 'emath_qns_eval_data_with_random.csv'
df_updated.to_csv(output_path_updated, index=False)

output_path_updated

'emath_qns_eval_data_with_random.csv'

# Generate output column for each query

In [ ]:
import pandas as pd
import sys
import os

from app.models import Message, QueryRequest
from app.main import query

# Load the CSV file
file_path = 'test/amath_qns_eval_data_with_random.csv'
df = pd.read_csv(file_path)

# Initialize the new column
df['output'] = ""

# Iterate over each row and populate the 'output' column
for index, row in df.iterrows():
    try:
        print(f"Processing row {index}...")
        # Create the QueryRequest object
        request = QueryRequest(
            user_query=[Message(role="user", content=row['question'])],
            subject="additional_mathematics",
            level=None,  # Use None for optional parameters
            exam_type=None,
            is_plain_text=True,  # Assuming plain text input for simplicity
            retrieved_documents=None,  # Assuming no pre-retrieved documents
        )

        # Call the query function
        response = query(request)

        # Extract the "question_text" field from the response and update the DataFrame
        question_text = response['response'].get('question_text', '')
        df.at[index, 'output'] = question_text

    except Exception as e:
        # Handle errors gracefully, log the error, and continue
        print(f"Error processing row {index}: {e}")
        df.at[index, 'output'] = "Error generating output"

Processing row 0...
Sending request to OpenAI API to get topic...
Sending request to OpenAI API to get sub-topic...
No matching text found in PDF - saving full pages
Saved combined pages 17 to 17 to temp/outram_secondary_school_elementary_mathematics_preliminary_exam_2024_1_20b.png
Found match on page 7. Saved to temp/xinmin_secondary_school_elementary_mathematics_preliminary_exam_2023_2_4e(i).png
Found match on page 7. Saved to temp/xinmin_secondary_school_elementary_mathematics_preliminary_exam_2023_2_4e(ii).png
Processing row 1...
Sending request to OpenAI API to get topic...
Sending request to OpenAI API to get sub-topic...
No matching text found in PDF - saving full pages
Saved combined pages 17 to 17 to temp/outram_secondary_school_elementary_mathematics_preliminary_exam_2024_1_20b.png
Found match on page 6. Saved to temp/xinmin_secondary_school_elementary_mathematics_preliminary_exam_2023_2_4b.png
Found match on page 6. Saved to temp/swiss_cottage_secondary_school_elementary_mat

In [ ]:
# Save the updated DataFrame to a new CSV file
output_path = 'test/amath_qns_eval_data_with_output.csv'
df.to_csv(output_path, index=False)

output_path

'test/emath_qns_eval_data_with_output.csv'